In [ ]:
#---#| default_exp constants.modification

# Modification information

The default modification TSV is stored in `alphabase/constants/const_files/modification.tsv`. Please check it to add more modifications. 

First, we load `modification.tsv` into `MOD_DF`. 

Then, we extract information of `MOD_CHEM` (dict), `MOD_MASS` (dict), `MOD_LOSS_MASS` (dict), `MOD_INFO_DICT` (dict) ... from `MOD_DF`. This step is done in `update_all_by_MOD_DF`.

All these steps are done by `load_mod_df`.

In [1]:
import os

import numpy as np
import pandas as pd

from alphabase.constants._const import CONST_FILE_FOLDER
import alphabase.constants.modification as modification
from alphabase.constants.modification import MOD_DF, calc_modification_mass, load_mod_df, calc_modloss_mass, \
    add_modifications_for_lower_case_AA, update_all_by_MOD_DF, MOD_INFO_DICT, add_new_modifications, MOD_Composition, \
    MOD_MASS, MOD_LOSS_IMPORTANCE

In [2]:
modification.MOD_DF

,mod_name,unimod_mass,unimod_avge_mass,composition,unimod_modloss,modloss_composition,classification,unimod_id,smiles,modloss_importance,mass,modloss_original,modloss
mod_name,,,,,,,,,,,,,
Acetyl@T,Acetyl@T,42.010565,42.0367,H(2)C(2)O(1),0.0,,Post-translational,1,,0.0,42.010565,0.0,0.0
Acetyl@Protein_N-term,Acetyl@Protein_N-term,42.010565,42.0367,H(2)C(2)O(1),0.0,,Post-translational,1,C(=O)C,0.0,42.010565,0.0,0.0
Acetyl@S,Acetyl@S,42.010565,42.0367,H(2)C(2)O(1),0.0,,Post-translational,1,,0.0,42.010565,0.0,0.0
Acetyl@C,Acetyl@C,42.010565,42.0367,H(2)C(2)O(1),0.0,,Post-translational,1,,0.0,42.010565,0.0,0.0
Acetyl@Any_N-term,Acetyl@Any_N-term,42.010565,42.0367,H(2)C(2)O(1),0.0,,Multiple,1,C(=O)C,0.0,42.010565,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
Ethyl@Protein_C-term,Ethyl@Protein_C-term,0.000000,0.0000,C(2)H(4),0.0,,User-added,0,OCC,0.0,28.031300,0.0,0.0
Cation:Na@Protein_C-term,Cation:Na@Protein_C-term,0.000000,0.0000,H(-1)Na(1),0.0,,User-added,0,O[Na],0.0,21.981944,0.0,0.0
Cation:K@Protein_C-term,Cation:K@Protein_C-term,0.000000,0.0000,H(-1)K(1),0.0,,User-added,0,O[K],0.0,37.955881,0.0,0.0


### With NIST elements, all calculated masses are the same as unimod_mass now.

In [3]:
#| hide
for mod, unimod_mass, mass in MOD_DF[['mod_name','unimod_mass','mass']].values:
    if abs(unimod_mass-mass) > 1e-5:
        print(f"{mod}: unimod mod={unimod_mass}, formula mass={mass}")

Delta:Hg(1)@C: unimod mod=201.970617, formula mass=201.9706434
Pro->(2S,4R)-4-fluoroproline@P: unimod mod=0.0, formula mass=17.9905781305
Pro->(2S,4S)-4fluoroproline@P: unimod mod=0.0, formula mass=17.9905781305
Pro->(2S)-1,3-thiazolidine-2-carboxylic_acid@P: unimod mod=0.0, formula mass=17.95642110994
Pro->(4R)-1,3-Thiazolidine-4-carboxylic_acid@P: unimod mod=0.0, formula mass=17.95642110994
Pro->(2S,4R)-4-hydroxyproline@P: unimod mod=0.0, formula mass=15.99491461957
Pro->(DL)-pipecolic_acid@P: unimod mod=0.0, formula mass=14.01565006446
Pro->3,4-Dehydro-L-proline@P: unimod mod=0.0, formula mass=-2.01565006446
Pro->(1S,3S,5S)-2-Azabicyclo[3.1.0]hexane-3-carboxylic_acid@P: unimod mod=0.0, formula mass=12.0
Pro->(1R,3S,5R)-2-Azabicyclo[3.1.0]hexane-3-carboxylic_acid@P: unimod mod=0.0, formula mass=12.0
Pro->(2S,3aS,7aS)-Octahydro-1H-indole-2-carboxylic_acid@P: unimod mod=0.0, formula mass=54.04695019338
Pro->(DL)-5-trifluoromethylproline@P: unimod mod=0.0, formula mass=67.98738445596
mT

# Mod site representation
* `site=0` refers to an N-term modification
* `site=-1` refers to a C-term modification
* `1<=site<=peplen` refers to a normal modification

For example: \_0A1B2C3D4E5F6G7H8I9J10K11\_-1

`calc_modification_mass` and `calc_modification_mass_sum` are base functions to calculate masses. But for large sets of peptides, we recommend to use `calc_mod_masses_for_same_len_seqs` instead of `calc_modification_mass`.

In [4]:
#| hide
seq = 'AGHCEWQMK'
mod_names = ['Acetyl@Protein_N-term', 'Carbamidomethyl@C', 'Oxidation@M']
mod_sites = [0, 4, 8]

assert np.allclose(
    calc_modification_mass(len(seq), mod_names, mod_sites),
    [42.01056468, 0, 0, 57.02146372, 0, 0, 0, 15.99491462, 0]
)

In [5]:
#| hide
mod_names = ['Oxidation@M', 'Phospho@S', 'Carbamidomethyl@C']
mod_sites = [0, 4, 8]

load_mod_df(modloss_importance_level=0)
assert np.allclose(
    calc_modloss_mass(10, mod_names, mod_sites, True),
    [63.99828592, 63.99828592, 63.99828592, 97.97689557, 97.97689557,
    97.97689557, 97.97689557, 97.97689557, 97.97689557]
)

load_mod_df(tsv=os.path.join(CONST_FILE_FOLDER, 'modification.tsv'), modloss_importance_level=1)
assert np.allclose(
    calc_modloss_mass(10, mod_names, mod_sites, True),
    [0, 0, 0, 97.97689557, 97.97689557,
    97.97689557, 97.97689557, 97.97689557, 97.97689557]
)

assert np.allclose(
    calc_modloss_mass(10, mod_names, mod_sites, False),
    [97.97689557, 97.97689557, 97.97689557,  0 ,  0,
        0,  0 ,  0,  0]
)

### Note that get_modloss_mass is a little bit time comsuming
`%timeit get_modloss_mass(10, mod_names, mod_sites, False)`

`Results (12 seconds in total): 12.6 µs ± 96.4 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)`

In [6]:
MOD_DF.drop_duplicates('classification')

,mod_name,unimod_mass,unimod_avge_mass,composition,unimod_modloss,modloss_composition,classification,unimod_id,smiles,modloss_importance,mass,modloss_original,modloss
mod_name,,,,,,,,,,,,,
Acetyl@T,Acetyl@T,42.010565,42.0367,H(2)C(2)O(1),0.000000,,Post-translational,1,,0.0,42.010565,0.000000,0.0
Acetyl@Any_N-term,Acetyl@Any_N-term,42.010565,42.0367,H(2)C(2)O(1),0.000000,,Multiple,1,C(=O)C,0.0,42.010565,0.000000,0.0
Acetyl@Y,Acetyl@Y,42.010565,42.0367,H(2)C(2)O(1),0.000000,,Chemical derivative,1,,0.0,42.010565,0.000000,0.0
Acetyl@R,Acetyl@R,42.010565,42.0367,H(2)C(2)O(1),0.000000,,Artefact,1,,0.0,42.010565,0.000000,0.0
ICAT-G@C,ICAT-G@C,486.251206,486.6253,H(38)C(22)N(4)O(6)S(1),0.000000,,Isotopic label,8,,0.0,486.251206,0.000000,0.0
Oxidation@G^Any_C-term,Oxidation@G^Any_C-term,15.994915,15.9994,O(1),0.000000,,Pre-translational,35,,0.0,15.994915,0.000000,0.0
Hex@C,Hex@C,162.052824,162.1406,H(10)C(6)O(5),0.000000,,Other glycosylation,41,,0.0,162.052823,0.000000,0.0
Hex@T,Hex@T,162.052824,162.1406,H(10)C(6)O(5),162.052824,H(10)C(6)O(5),O-linked glycosylation,41,,0.5,162.052823,162.052823,0.0
Hex@N,Hex@N,162.052824,162.1406,H(10)C(6)O(5),162.052824,H(10)C(6)O(5),N-linked glycosylation,41,,0.5,162.052823,162.052823,0.0


## We can update modification list for differet requirements, for example:

In [7]:
add_modifications_for_lower_case_AA()
modification.MOD_DFMOD_DF

,mod_name,unimod_mass,unimod_avge_mass,composition,unimod_modloss,modloss_composition,classification,unimod_id,smiles,modloss_importance,mass,modloss_original,modloss,lower_case_AA
mod_name,,,,,,,,,,,,,,
Acetyl@T,Acetyl@T,42.010565,42.0367,H(2)C(2)O(1),0.0,,Post-translational,1,,0.0,42.010565,0.0,0.0,False
Acetyl@Protein_N-term,Acetyl@Protein_N-term,42.010565,42.0367,H(2)C(2)O(1),0.0,,Post-translational,1,C(=O)C,0.0,42.010565,0.0,0.0,False
Acetyl@S,Acetyl@S,42.010565,42.0367,H(2)C(2)O(1),0.0,,Post-translational,1,,0.0,42.010565,0.0,0.0,False
Acetyl@C,Acetyl@C,42.010565,42.0367,H(2)C(2)O(1),0.0,,Post-translational,1,,0.0,42.010565,0.0,0.0,False
Acetyl@Any_N-term,Acetyl@Any_N-term,42.010565,42.0367,H(2)C(2)O(1),0.0,,Multiple,1,C(=O)C,0.0,42.010565,0.0,0.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"Pro->3,4-Dehydro-L-proline@p","Pro->3,4-Dehydro-L-proline@p",0.000000,0.0000,H(-2),0.0,,User-added,0,C1C=CC(N1([Xe]))C(=O)[Rn],0.0,-2.015650,0.0,0.0,True
"Pro->(1S,3S,5S)-2-Azabicyclo[3.1.0]hexane-3-carboxylic_acid@p","Pro->(1S,3S,5S)-2-Azabicyclo[3.1.0]hexane-3-ca...",0.000000,0.0000,C(1),0.0,,User-added,0,[C@H]12N([Xe])[C@@H](C[C@@H]2C1)C(=O)[Rn],0.0,12.000000,0.0,0.0,True
"Pro->(1R,3S,5R)-2-Azabicyclo[3.1.0]hexane-3-carboxylic_acid@p","Pro->(1R,3S,5R)-2-Azabicyclo[3.1.0]hexane-3-ca...",0.000000,0.0000,C(1),0.0,,User-added,0,[C@@H]12N([Xe])[C@@H](C[C@H]2C1)C(=O)[Rn],0.0,12.000000,0.0,0.0,True


In [8]:

modification.MOD_DF = MOD_DF[
    (modification.MOD_DF['classification'].isin(['Post-translational','O-linked glycosylation','AA substitution','Multiple','Non-standard residue','Pre-translational']))
    & modification.MOD_DF['lower_case_AA']
] # we only need PTMs
update_all_by_MOD_DF()
# MOD_INFO_DICT is also updated
pd.DataFrame().from_dict(MOD_INFO_DICT, orient='index')

/var/folders/6r/8h59xkv90qs2dyy8mdn_ch840000gn/T/ipykernel_50785/1388563684.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  modification.MOD_DF = MOD_DF[


""


In [12]:
#| hide
load_mod_df()
add_modifications_for_lower_case_AA()
MOD_DF = modification.MOD_DF
MOD_DF = MOD_DF[
    MOD_DF['lower_case_AA']
] # we only need PTMs
assert MOD_DF['mod_name'].apply(lambda x: x[x.find('@')+1].islower()).all()

In [13]:
#| hide
prev_value = (modification.MOD_DF.classification=='User-added').sum()
add_new_modifications([
    ("Hello@S","H(2)"),
    ("World@S","O(10)","O(3)")
])
assert (modification.MOD_DF.classification=='User-added').sum() - prev_value == 2
assert 'Hello@S' in modification.MOD_DF.mod_name
assert 'World@S' in modification.MOD_DF.mod_name
assert modification.MOD_DF.loc['World@S','modloss'] > 0
assert modification.MOD_DF.loc['World@S','modloss_importance'] > 0
assert 'Hello@S' in MOD_Composition
assert 'World@S' in MOD_MASS

In [15]:
add_new_modifications({
    "Hi@S":{'composition':"H(2)"},
    "AlphaX@S":{'composition':"O(10)",'modloss_composition':"O(3)"}
})
assert (modification.MOD_DF.classification=='User-added').sum() - prev_value == 4
assert 'Hi@S' in modification.MOD_DF.mod_name
assert 'Hi@S' in modification.MOD_DF.index
assert 'AlphaX@S' in modification.MOD_DF.mod_name
assert 'AlphaX@S' in modification.MOD_DF.index
assert modification.MOD_DF.loc['AlphaX@S','modloss'] > 0
assert modification.MOD_DF.loc['AlphaX@S','modloss_importance'] > 0
assert 'Hi@S' in MOD_Composition
assert 'AlphaX@S' in MOD_MASS
assert 'AlphaX@S' in MOD_LOSS_IMPORTANCE
assert modification.MOD_DF.loc['AlphaX@S','unimod_mass'] == 0

In [ ]:
MOD_DF

,mod_name,unimod_mass,unimod_avge_mass,composition,unimod_modloss,modloss_composition,classification,unimod_id,modloss_importance,mass,modloss_original,modloss,lower_case_AA
mod_name,,,,,,,,,,,,,
Acetyl@T,Acetyl@T,42.010565,42.0367,H(2)C(2)O(1),0.0,,Post-translational,1.0,0.000000e+00,42.010565,0.0,0.000000,False
Acetyl@Protein_N-term,Acetyl@Protein_N-term,42.010565,42.0367,H(2)C(2)O(1),0.0,,Post-translational,1.0,0.000000e+00,42.010565,0.0,0.000000,False
Acetyl@S,Acetyl@S,42.010565,42.0367,H(2)C(2)O(1),0.0,,Post-translational,1.0,0.000000e+00,42.010565,0.0,0.000000,False
Acetyl@C,Acetyl@C,42.010565,42.0367,H(2)C(2)O(1),0.0,,Post-translational,1.0,0.000000e+00,42.010565,0.0,0.000000,False
Acetyl@Any_N-term,Acetyl@Any_N-term,42.010565,42.0367,H(2)C(2)O(1),0.0,,Multiple,1.0,0.000000e+00,42.010565,0.0,0.000000,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
GlyGly@k,GlyGly@k,114.042927,114.1026,H(6)C(4)N(2)O(2),0.0,,Post-translational,121.0,1.000000e+06,114.042927,0.0,0.000000,True
Hello@S,Hello@S,0.000000,0.0000,H(2),0.0,,User-added,0.0,0.000000e+00,2.015650,0.0,0.000000,0
World@S,World@S,0.000000,0.0000,O(10),0.0,O(3),User-added,0.0,1.000000e+100,159.949146,0.0,47.984744,0
